## DSN  : Predicitve Model Notebook [Using Light GBM]
**Author**:🧕🏿 Hasanat Owoseni \
**Date** : 14th October, 2019

In [20]:
import numpy as np
import pandas as pd
import pickle
%matplotlib inline

In [21]:
test_df = pd.read_csv('cleaned_test.csv')
train_df = pd.read_csv('cleaned_train.csv')

In [22]:
test_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 17 columns):
trainings_attended          38312 non-null int64
targets_met                 38312 non-null int64
previous_award              38312 non-null int64
training_score_average      38312 non-null int64
no_of_previous_employers    38312 non-null int64
promoted_or_not             38312 non-null int64
age                         38312 non-null int64
last_performance_score      38312 non-null float64
working_for                 38312 non-null int64
qualification               38312 non-null int64
channel_of_recruitment      38312 non-null int64
is_female                   38312 non-null int64
schooled_abroad             38312 non-null int64
is_married                  38312 non-null int64
was_disciplined             38312 non-null int64
was_moved                   38312 non-null int64
division                    38312 non-null int64
dtypes: float64(1), int64(16)
memory usage: 5.0 MB
<clas

In [23]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

In [24]:
columns = [column for column in test_df.columns if column != 'employeeno']
test_x = test_df[columns].values.astype('float')

test_empid = df_test['EmployeeNo']
#what test x should be 
s_test_x = test_df.loc[: ,'trainings_attended':].values.astype('float')

In [25]:
columns = [column for column in train_df.columns if column != ('promoted_or_not' ) and column !=( 'employeeno')]

X = train_df[columns].values.astype('float')
s_train = train_df[columns]

y = train_df['promoted_or_not']
empid = df_train['EmployeeNo']

In [26]:
print(test_x.shape)
print(X.shape)
print(s_test_x.shape)
s_train.info()

(16496, 16)
(38312, 16)
(16496, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 16 columns):
trainings_attended          38312 non-null int64
targets_met                 38312 non-null int64
previous_award              38312 non-null int64
training_score_average      38312 non-null int64
no_of_previous_employers    38312 non-null int64
age                         38312 non-null int64
last_performance_score      38312 non-null float64
working_for                 38312 non-null int64
qualification               38312 non-null int64
channel_of_recruitment      38312 non-null int64
is_female                   38312 non-null int64
schooled_abroad             38312 non-null int64
is_married                  38312 non-null int64
was_disciplined             38312 non-null int64
was_moved                   38312 non-null int64
division                    38312 non-null int64
dtypes: float64(1), int64(15)
memory usage: 4.7 MB


In [27]:
#train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(30649, 16) (30649,)
(7663, 16) (7663,)


--------------------- 

In [28]:
import lightgbm as lgb

In [29]:
lgb_model = lgb.LGBMClassifier(n_estimators=1000, max_depth=3, learning_rate=0.01, random_state=1)

In [30]:
from sklearn.metrics import roc_auc_score

In [31]:
lgb_model.fit(X_train, y_train)
pred_lgb = lgb_model.predict(X_test)
print(roc_auc_score(pred_lgb, y_test))

0.8868568252330824


In [32]:
lgb_model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=3,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [33]:
with open('lgbm.pkl', 'wb') as file:
    pickle.dump(lgb_model, file)

In [34]:
ypred2 = lgb_model.predict(test_x)
ypred2[0:5]  # showing first 5 predictions
y_test[0:5]

18116    0
25421    0
10129    0
18814    0
27359    0
Name: promoted_or_not, dtype: int64

In [35]:
for i in range(0,16496):
    if ypred2[i]>=.5:       # setting threshold to .5
        ypred2[i]=1.0
    else:  
        ypred2[i]=0.0

from sklearn.metrics import accuracy_score

In [36]:
ypred2[0:5]  # showing first 5 predictions

array([0, 0, 0, 0, 0], dtype=int64)

In [37]:
df_submission = pd.DataFrame({'employeeno':df_test['EmployeeNo'], 'promoted_or_not':ypred2.astype('int64')})
df_submission.to_csv('set.csv', index = False)

In [38]:
from sklearn.metrics import roc_auc_score